In [6]:
import pm4py
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import sorting

In [7]:
LOG_PATH = "bpi-chall.xes"

elog = pm4py.read_xes(LOG_PATH)

if isinstance(elog, pd.DataFrame):
    elog = log_converter.apply(elog, variant=log_converter.Variants.TO_EVENT_LOG)

elog = sorting.sort_timestamp(elog, timestamp_key="time:timestamp")

print(type(elog))
print(f"Number of cases: {len(elog)}")

parsing log, completed traces :: 100%|██████████| 31509/31509 [00:33<00:00, 952.21it/s] 


<class 'pm4py.objects.log.obj.EventLog'>
Number of cases: 31509
